## Global Wheat Detection: Can you help identify wheat heads using image analysis?

> Open up your pantry and you’re likely to find several wheat products. Indeed, your morning toast or cereal may rely upon this common grain. Its popularity as a food and crop makes wheat widely studied. To get large and accurate data about wheat fields worldwide, plant scientists use image detection of "wheat heads"—spikes atop the plant containing grain. These images are used to estimate the density and size of wheat heads in different varieties. Farmers can use the data to assess health and maturity when making management decisions in their fields.

![](https://storage.googleapis.com/kaggle-media/competitions/UofS-Wheat/descriptionimage.png)

> However, accurate wheat head detection in outdoor field images can be visually challenging. There is often overlap of dense wheat plants, and the wind can blur the photographs. Both make it difficult to identify single heads. Additionally, appearances vary due to maturity, color, genotype, and head orientation. Finally, because wheat is grown worldwide, different varieties, planting densities, patterns, and field conditions must be considered. Models developed for wheat phenotyping need to generalize between different growing environments. Current detection methods involve one- and two-stage detectors (Yolo-V3 and Faster-RCNN), but even when trained with a large dataset, a bias to the training region remains.

>In this competition, you’ll detect wheat heads from outdoor images of wheat plants, including wheat datasets from around the globe. Using worldwide data, you will focus on a generalized solution to estimate the number and size of wheat heads. To better gauge the performance for unseen genotypes, environments, and observational conditions, the training dataset covers multiple regions. You will use more than 3,000 images from Europe (France, UK, Switzerland) and North America (Canada). The test data includes about 1,000 images from Australia, Japan, and China.

In [ ]:
import os
import cv2
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
!ls ../input/global-wheat-detection

In [ ]:
data_dir = '../input/global-wheat-detection'

In [ ]:
train = pd.read_csv(f'{data_dir}/train.csv')
train.sample(10)

> The CSV data is simple - the image ID matches up with the filename of a given image, and the width and height of the image are included, along with a bounding box. There is a row in train.csv for each bounding box. Not all images have bounding boxes.

In [ ]:
print("Shape of train.csv: ", train.shape)
print("Unique image_ids in train.csv: ", len(train.image_id.unique()))

Since there is a row for each bounding box in train.csv, the shape is `147793` but when we consider just the unique image_ids, the number reduces down to `3373`.
Now let's take a look at the images present in `train` folder

In [ ]:
train_images = glob.glob(f'{data_dir}/train/*')
len(train_images), train_images[0]

There are `3422` images in `train` folder, but the number of unique image_ids in `train.csv` is just `3373`. This means that there are images in train-folder which doesn't have any kind of wheat present in it. Let's take a look at some of the images with bouding box plotted on them

In [ ]:
def overlay_box(im, box, rgb, stroke=1):
    """
    Method to overlay single box on image

    """
    # --- Convert coordinates to integers
    box = box[1:-1].split(',')
    box = [float(b) for b in box]
    box = [int(b) for b in box]
    
    # --- Extract coordinates
    x1, y1, width, height = box # xmin, ymin, width, height
    y2 = y1 + height
    x2 = x1 + width

    im[y1:y1 + stroke, x1:x2] = rgb
    im[y2:y2 + stroke, x1:x2] = rgb
    im[y1:y2, x1:x1 + stroke] = rgb
    im[y1:y2, x2:x2 + stroke] = rgb

    return im

In [ ]:
def draw_bbox(image, bbox):
    for box in bbox:
        rgb = np.floor(np.random.rand(3) * 1024).astype('int')
        image = overlay_box(im=image, box=box, rgb=rgb, stroke=6)
    return image

In [ ]:
fig=plt.figure(figsize=(25, 25))
columns = 3
rows = 3
for i in range(1, columns*rows +1):
    name = train_images[i].split('/')[-1]
    img = cv2.imread(train_images[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    fig.add_subplot(rows, columns, i)
    bbox = train.loc[train['image_id']==name.split('.')[0], 'bbox'].values
    img = draw_bbox(img, bbox)
    plt.imshow(img)
    plt.title(name)
    plt.axis('off')
plt.show()

The images looks pretty good and each image has numerous wheat heads and detecting the correct number of wheat heads along with bounding-box coordinates might be a challenging(and fun) task in this competition. Let's explore the count(number) of bounding boxes for one-particular image_id.

In [ ]:
df_train = train[['image_id', 'bbox']]
df_counted = df_train.groupby('image_id').count()
df_counted = df_counted.rename(columns={'bbox': 'count'})
df_counted.head()

In [ ]:
df_counted['count'].mean()

So, there are roughly 43 bounding boxes for each image_id. The overall distribution is plotted below

In [ ]:
plt.figure(figsize=(25, 20))
df_counted['count'].value_counts().plot(kind='bar');
plt.ylabel('number of images', fontsize=12);
plt.xlabel('number of bboxes', fontsize=12);

Now, let's take a look at images in the test-folder, for which we need to make predictions
>You are attempting to predict bounding boxes around each wheat head in images that have them. If there are no wheat heads, you must predict no bounding boxes.

In [ ]:
sub = pd.read_csv(f'{data_dir}/sample_submission.csv')
sub.head()

In [ ]:
sub.shape

In [ ]:
test_images = glob.glob(f'{data_dir}/test/*')
len(test_images), test_images[0]

Interestingly, there are just 10-images available to us for making prediction as most of the test-images are hidden. I guess the organizers have their own reasons for hiding most of the test-images. My best guess would be that they want to prevent pseudo-labeling of test-images 

In [ ]:
fig=plt.figure(figsize=(25, 25))
columns = 3
rows = 3
for i in range(1, columns*rows +1):
    name = test_images[i].split('/')[-1]
    img = cv2.imread(test_images[i])
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
    plt.title(name)
    plt.axis('off')
plt.show()

This will be a fun competition to participate as this is a detection-based competition. It would be a nice opportunity to learn(and use) the recent advancements in Image-Detection domain such as [EfficientDet](https://arxiv.org/abs/1911.09070) and [YoloV4](https://arxiv.org/abs/2004.10934)
> Expect a lot of computer-vision loving Kagglers to join as the dataset is not that huge compared to other ongoing vision competitions. 